Notes:
- nltk: stop words package that's already implemented
- Built-in similarity function for spacy, needs to be called on an nlp-type version of a string object
- Spacy type objects & dont want to tokenize things
- texty (TF-IDF vectorizer and spacy had a baby)

In [1]:
import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m pip install nltk
!{sys.executable} python -m spacy download en_core_web_sm

/Users/jericahuang/CS4300_Flask_template/venv/bin/python: can't open file 'python': [Errno 2] No such file or directory


In [2]:
# the imports used in A2
import re
# import json
from glob import glob
import os
from io import StringIO
from itertools import groupby
import pickle

import numpy as np
# import bs4
# %matplotlib inline
# import matplotlib.pyplot as plt
# Imports that might help with various functionality
import functools
import operator

# Additional imports from A3
from __future__ import print_function
import math
from collections import defaultdict
# from nltk.tokenize import TreebankWordTokenizer
# import Levenshtein  # package python-Levenshtein

# Additional imports from A5
import nltk
# nltk.download()
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jericahuang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [71]:
# SPACY package
import spacy
# nlp = spacy.load('en')
nlp = spacy.load('en_core_web_md')

# Import Dataset

In [55]:
# import the json
import json
with open("data.json", "r") as f:
    women_summaries = json.load(f)
    
# PEGAH (From A5)
print("Each woman's entry is a dictionary with the following keys...")
print(women_summaries[0].keys())

deduped_women_summaries = {}
for woman in women_summaries:
    if woman['name'] not in deduped_women_summaries:
        deduped_women_summaries[woman['name']] = woman['summary']
    else:
        deduped_women_summaries[woman['name']] += "\n" + woman['summary']        

Each woman's entry is a dictionary with the following keys...
[u'url', u'views', u'name', u'summary']


In [5]:
# REMOVED FROM SPACY VERSION
# creates a list of women to keep as a global variable
women_names = list()
for i in range(len(women_summaries)):
    name = women_summaries[i]['name']
    end_index = name.find('(')
    if end_index != -1 and name[: (end_index-1)] not in women_names :
        women_names.append(name[: (end_index-1)])
    elif end_index == -1 and name not in women_names :
        women_names.append(name)
        
# print(women_names)
# women_names is a list of the names of women in order of the JSON file

# Deduplicate

In [56]:
deduped_women = [{'summary': summary, 'name': name} for (name, summary) in list(deduped_women_summaries.items())]
import pickle
pickle.dump(deduped_women, open( "deduped_women.pickle", "wb" ), protocol=2)

In [57]:
for entry in deduped_women:
    if entry["name"].find("(")>0:
        entry["name"] = entry["name"][:entry["name"].find("(")-1]
        
pickle.dump(deduped_women, open( "deduped_women.pickle", "wb" ), protocol=2)

# Spacy Dictionary (Commented out for now)

In [8]:
from __future__ import unicode_literals

In [9]:
# Builds a dictionary that maps woman to the list of tokens associated with her
# {'woman1' : ['token1', 'token2', ... 'tokenn']}
from __future__ import unicode_literals

def build_spacy_token_dictionary(input_summaries, input_women_names):
    # Builds a dictionary that maps each woman to their tokenized words (yes we've already done this but we want the spacy version)
    women_token_dictionary = dict()
    
    for i in range(len(input_women_names)):
        name = input_women_names[i]
        summary = input_summaries[i]['summary']
    
        index_was = summary.find('was ')
        index_is = summary.find('is ')
        index_currently = summary.find('currently ')
        if (index_was == -1 and index_is == -1):
            summary2 = summary[index_currently:]
        elif (index_was == -1 and index_currently == -1):
            summary2 = summary[index_is:]
        elif (index_is == -1 and index_currently == -1):
            summary2 = summary[index_was:]
        else :
            index_min = min(index_is, index_was)
            summary2 = summary[index_min:]
            
        # lowercase long string of summary
        summary_lower = summary2.lower()
        # doc object type
        summary_nlp = nlp(summary_lower)
        # type list
        summary_list = [token.text for token in summary_nlp]
        # type list, but without stop words
        summary_token_filtered = [w for w in summary_list if w not in stopwords.words('english')]
        
        # convert to a string separated by spaces
        summary_str = " ".join(summary_token_filtered) 
        summary_nlp = nlp(summary_str)
        women_token_dictionary[name] = summary_nlp
    
#     print(women_token_dictionary)
    return women_token_dictionary

In [ ]:
# COMMENTED OUT BECAUSE TOKENIZER DOES THIS
women_token_dictionary = build_spacy_token_dictionary(women_summaries, women_names)
# # print(women_token_dictionary)
pickle.dump(women_token_dictionary, open( "women_token_dictionary.pickle", "wb" ), protocol=2)

# TF-IDF Vectorizer
#### Includes pickling the vectorizer and the TF-IDF matrix

In [72]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words="english", max_df = 0.95)
matx = vectorizer.fit_transform(map(lambda x: x["summary"], deduped_women))

import pickle
with open("myvectorizer.pickle", "wb+") as f:
    pickle.dump(vectorizer, f)
    pickle.dump(matx, f)

# Cosine Similarity

In [12]:
# query = "is a mother"
# q_vec = vectorizer.transform([query])
# sim_doc_scores = cosine_similarity(q_vec, matx)
# # apply weighting here
# # print(sum(sim_doc_scores.flatten()))
# sim_docs = np.argsort(sim_doc_scores.flatten())[::-1]
# sim_docs

# JUST KIDDING!! WE DO THIS IN THE FUNCTION BELOW

# Spacy Similarity Matrix

In [59]:
def build_sim_matrix(input_token_dictionary, input_len_names, input_women_names):
    # Builds matrix using spacy that stores each woman's cosine similarity to each other
    
    sim_matrix = np.zeros(shape = (input_len_names, input_len_names))
    
    for woman1 in input_token_dictionary:
        for woman2 in input_token_dictionary:
            woman1_index = input_women_names.index(woman1)
            woman2_index = input_women_names.index(woman2)
#             print(input_token_dictionary[woman1])
#             print(input_token_dictionary[woman2])
            sim_matrix[woman1_index][woman2_index] = input_token_dictionary[woman1].similarity(input_token_dictionary[woman2])
    np.fill_diagonal(sim_matrix, 0)
    
    return sim_matrix
        

In [60]:
women2women_cosine_sim_matrix = build_sim_matrix(women_token_dictionary, len(women_names), women_names)

In [61]:
pickle.dump(women2women_cosine_sim_matrix, open( "women2women_cosine_sim_matrix.pickle", "wb" ), protocol=2)
print(women2women_cosine_sim_matrix[1][0])

0.914916261736464


In [62]:
print(women2women_cosine_sim_matrix)

[[0.         0.91491626 0.91129139 ... 0.90106309 0.86203536 0.83305594]
 [0.91491626 0.         0.88929614 ... 0.91033143 0.87483623 0.90432014]
 [0.91129139 0.88929614 0.         ... 0.8984671  0.88708984 0.82265488]
 ...
 [0.90106309 0.91033143 0.8984671  ... 0.         0.89060047 0.85379527]
 [0.86203536 0.87483623 0.88708984 ... 0.89060047 0.         0.88096003]
 [0.83305594 0.90432014 0.82265488 ... 0.85379527 0.88096003 0.        ]]


In [63]:
import operator

# Code taken from A5
def get_ranked_women(input_woman, input_sim_matrix, input_women_names) :
#     print(input_sim_matrix)
    '''Return sorted rankings (most to least similar) of women as 
        a list of two-element tuples, where the first element is the 
        woman's name and the second element is the similarity score
    '''
    
    # Get index from woman's name
    idx = input_women_names.index(input_woman)
    
    # Get list of similarity scores for woman
    score_lst = input_sim_matrix[idx]
    women_score_lst = [(input_women_names[index], score) for index, score in enumerate(score_lst)]
    
    # Do not account for woman herself in ranking
    women_score_lst = women_score_lst[:idx] + women_score_lst[idx+1:]
    
    # Sort rankings by score (most similar to least similar)
    women_score_lst = sorted(women_score_lst, key=lambda x: -x[1])
    
    # Only returning top 5!
    return women_score_lst[:5]

In [64]:
# Builds a 'top 5 most similar' list of woman for every woman
def create_top_5_dict_women(input_sim_mat, num_women, input_women_names):
    women_top5 = {}
    
    # Loop through each woman
    for woman in range(num_women):
        similarity_list = get_ranked_women(input_women_names[woman], input_sim_mat, input_women_names)
        similarity_list = [x[0] for x in similarity_list]
        women_top5[input_women_names[woman]] = similarity_list
        
    return women_top5
            

In [65]:
top5_dict_women2women = create_top_5_dict_women(women2women_cosine_sim_matrix, len(women_names), women_names)

In [1]:
import pickle
top_5_dict_women = pickle.load( open( "top_5_dict_women.pickle", "rb" ) )
upper_to_official = {}
for key in top_5_dict_women:
    upper_to_official[key.upper()]=key

upper_to_official

{u'SHIRLEY MCBAY': u'Shirley McBay',
 u'ORNA GRUMBERG': u'Orna Grumberg',
 u'MALIKA ZEGHAL': u'Malika Zeghal',
 u'MONICA WILSON': u'Monica Wilson',
 u'OLIVE SCOTT': u'Olive Scott',
 u'SEETHA COLEMAN-KAMMULA': u'Seetha Coleman-Kammula',
 u'GINA ATHENA ULYSSE': u'Gina Athena Ulysse',
 u'HAGIT ATTIYA': u'Hagit Attiya',
 u'BRIT SOLLI': u'Brit Solli',
 u'ANU\u0160KA FERLIGOJ': u'Anu\u0161ka Ferligoj',
 u'PENNY SACKETT': u'Penny Sackett',
 u'MARY BETH RUSKAI': u'Mary Beth Ruskai',
 u"FRANCES D'SOUZA, BARONESS D'SOUZA": u"Frances D'Souza, Baroness D'Souza",
 u'HUA ELEANOR YU': u'Hua Eleanor Yu',
 u'LEONA WOODS': u'Leona Woods',
 u'AYTEN AYDIN': u'Ayten Ayd\u0131n',
 u'JEAN E. SAMMET': u'Jean E. Sammet',
 u'REGINA S. BURACHIK': u'Regina S. Burachik',
 u'NINA GAGEN-TORN': u'Nina Gagen-Torn',
 u'MARIA CHUDNOVSKY': u'Maria Chudnovsky',
 u'KIM PLOFKER': u'Kim Plofker',
 u'CHIU-CHU MELISSA LIU': u'Chiu-Chu Melissa Liu',
 u'KARIMAT EL-SAYED': u'Karimat El-Sayed',
 u'ROSALIND MOSS': u'Rosalind Moss',

In [2]:
pickle.dump(upper_to_official, open( "upper_to_official.pickle", "wb" ), protocol=2)

# Query output for Spacy (Commented out for now)

In [68]:
# SPACY OUTPUT RETURN -- commented out for now
# def return_query(input_query, input_women_token_dictionary):
#     # Given a query string, this function will return a list of women that are most similar to the query
    
#     query = nlp(input_query)
#     results = [] # List that will eventually store name, similarity, popularity, and ultimate RANKING
    
#     # Ranking will be defined by: (1/popularity) + (0.5*similarity) + entity weighting
    
#     for woman in input_women_token_dictionary:
#         similarity = query.similarity(input_women_token_dictionary[woman])
#         if similarity > 0.2: # Arbitrary number, can play around with with
#             results.append((woman, similarity, similarity)) # Currently similarity will act as the ranking
            
#     results.sort(key=lambda x: x[2]) # Ascending order of rankings
#     results.reverse() # Descending order

# Query output for Cossim

In [73]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pickle

# Load from the pickle
pkl_file = open('myvectorizer.pickle', 'rb')
vectorizer = pickle.load(pkl_file)
matx = pickle.load(pkl_file)

def return_query(vectorizer, matx, query, data_dict):
    q_vec = vectorizer.transform([query])
    sim_doc_scores = cosine_similarity(q_vec, matx)
#     print(sim_doc_scores)
    # Apply weighting here!
    sim_docs = np.argsort(sim_doc_scores.flatten())[::-1]
    return_docs = []

    for hit in sim_docs:
        if sim_doc_scores[0][hit] > 0:
#             print(sim_doc_scores[0][hit])
            return_docs.append(data_dict[hit])
    if len(return_docs)>30:
        return_docs = return_docs[:30]
    return return_docs

In [77]:
return_query(vectorizer, matx, "is a french biologist", deduped_women)

[{u'name': u'Germaine Tillion',
  u'summary': u'Germaine Tillion (30 May 1907 \u2013 18 April 2008) was a French ethnologist, best known for her work in Algeria in the 1950s on behalf of the French government. A member of the French resistance, she spent time in the Ravensbr\xfcck concentration camp.'},
 {u'name': u'Valerie Mizrahi',
  u'summary': u'Valerie Mizrahi (born 1958) is a South African molecular biologist.'},
 {u'name': u'Lucie Randoin',
  u'summary': u'Lucie Gabrielle Randoin (11 May 1888 \u2013 13 September 1960) was a French biologist, nutritionist, and hygienist. She was made a commander of the Legion of Honour in 1958 and is known for her research on vitamins.'},
 {u'name': u'Yvonne Choquet-Bruhat',
  u'summary': u'Yvonne Choquet-Bruhat (French: [b\u0281y.a]; born 29 December 1923 in Lille) is a French mathematician and physicist. She was the first woman to be elected to the Acad\xe9mie des Sciences Fran\xe7aise ("French Academy of Sciences") and is a Grand Officier of t

# Testing Spacy

In [20]:
# doc1 = nlp(deduped_women[0]['summary'])
# # print(doc1)
# doc2 = nlp(deduped_women[1]['summary'])
# # print(doc2)

# arr = np.column_stack((doc1.vector,doc2.vector))

# doc3 = nlp(u'is a cinematographer')


# # print(deduped_women[0]['summary'])
# # print(doc.vector)
# # assert doc.vector.dtype == 'float32'
# # len(doc.vector)
# sim1 = np.dot(doc1.vector, doc3.vector)/(np.linalg.norm(doc1.vector)*np.linalg.norm(doc3.vector)) # cinema to query
# sim2 = np.dot(doc2.vector, doc3.vector)/(np.linalg.norm(doc2.vector)*np.linalg.norm(doc3.vector)) # physicist to query

# # print(doc1.similarity(doc3))
# # print(doc2.similarity(doc3))

# print(doc1)
# print(sim1)
# print()
# print(doc2)
# print(sim2)

# Making our Spacy Matrix

## <span style="color:red">Important!</span>
The next block takes a while to run, but it's been pickled! Don't run the next 2 code blocks. Instead, just compile spacy_sim and run the block after it.

In [21]:
# DO NOT RUN!!
spacy_array = np.zeros((len(deduped_women),300))
for index, woman in enumerate(deduped_women):
#     print(index)
    wom_vec = nlp(woman['summary']).vector
    if len(wom_vec) != 300:
        wom_vec = np.zeros(300)
    spacy_array[index] = wom_vec

In [22]:
# pickle here
# DON'T RUN THIS EITHER!
with open("spacyarray.pickle", "wb+") as f:
    pickle.dump(spacy_array, f)

In [9]:
def spacy_sim(spacy_mat, query, data_dict):
    '''
    spacy_mat: numpy array of documents by spacy vectors
    
    query: query string
    
    RETURNS
    
    '''
    q = nlp(query)
    print(q)
    sim_scores = np.dot(spacy_mat, q.vector)/(np.linalg.norm(spacy_mat)*np.linalg.norm(q.vector))
    sim_docs = np.argsort(sim_scores)[::-1]
    
    return_docs = []
    for hit in sim_docs[0:30]:
        return_docs.append(data_dict[hit])
    return return_docs

In [10]:
pkl_file_spacy = open('spacyarray.pickle', 'rb')
spacy_array = pickle.load(pkl_file_spacy)

# spacy_sim(spacy_array, "won the nobel prize", deduped_women)

# Combining our sim measures
Spacy similarity scores + cosine similarity scores

Next changes
* Have our similarity function return the sim_scores numpy array, and then have another function that uses our weighting algorithm to return the top 30
* Decide on TWE we want to use views in weighting if we're going to do a sort-by feature